In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [7]:
timeseries = np.array([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                       [0.1**3, 0.2**3, 0.3**3, 0.4**3, 0.5**3, 0.6**3, 0.7**3, 0.8**3, 0.9**3]]).transpose()

timesteps = timeseries.shape[0]
n_features = timeseries.shape[1]
timeseries

array([[0.1  , 0.001],
       [0.2  , 0.008],
       [0.3  , 0.027],
       [0.4  , 0.064],
       [0.5  , 0.125],
       [0.6  , 0.216],
       [0.7  , 0.343],
       [0.8  , 0.512],
       [0.9  , 0.729]])

In [6]:
def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

In [14]:
timesteps = 3
X, y = temporalize(X = timeseries, y = np.zeros(len(timeseries)), lookback = timesteps)

In [17]:
n_features = 2
X = np.array(X)

In [19]:
X = X.reshape(X.shape[0], timesteps, n_features)

In [20]:
X

array([[[0.3  , 0.027],
        [0.4  , 0.064],
        [0.5  , 0.125]],

       [[0.4  , 0.064],
        [0.5  , 0.125],
        [0.6  , 0.216]],

       [[0.5  , 0.125],
        [0.6  , 0.216],
        [0.7  , 0.343]],

       [[0.6  , 0.216],
        [0.7  , 0.343],
        [0.8  , 0.512]],

       [[0.7  , 0.343],
        [0.8  , 0.512],
        [0.9  , 0.729]]])

# Normal LSTM

In [21]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3, 128)            67072     
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 64)             49408     
_________________________________________________________________
lstm_3 (LSTM)                (None, 3, 64)             33024     
_________________________________________________________________
lstm_4 (LSTM)                (None, 3, 128)            98816     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 3, 2)              258       
Total params: 248,578
Trainable params: 248,578
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X, X, epochs=1, batch_size=5, verbose=0)
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))

Instructions for updating:
Use tf.cast instead.
---Predicted---
[[[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]

 [[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]

 [[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]

 [[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]

 [[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]]
---Actual---
[[[0.3   0.027]
  [0.4   0.064]
  [0.5   0.125]]

 [[0.4   0.064]
  [0.5   0.125]
  [0.6   0.216]]

 [[0.5   0.125]
  [0.6   0.216]
  [0.7   0.343]]

 [[0.6   0.216]
  [0.7   0.343]
  [0.8   0.512]]

 [[0.7   0.343]
  [0.8   0.512]
  [0.9   0.729]]]


# LSTM with Encoder Decoder 

In [23]:
model1 = Sequential()
model1.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model1.add(LSTM(64, activation='relu', return_sequences=False))
model1.add(RepeatVector(timesteps))
model1.add(LSTM(64, activation='relu', return_sequences=True))
model1.add(LSTM(128, activation='relu', return_sequences=True))
model1.add(TimeDistributed(Dense(n_features)))
model1.compile(optimizer='adam', loss='mse')
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 3, 128)            67072     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                49408     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 64)             0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 3, 64)             33024     
_________________________________________________________________
lstm_8 (LSTM)                (None, 3, 128)            98816     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 3, 2)              258       
Total params: 248,578
Trainable params: 248,578
Non-trainable params: 0
________________________________________________

In [24]:
model1.fit(X, X, epochs=1, batch_size=5, verbose=0)
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))

---Predicted---
[[[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]

 [[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]

 [[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]

 [[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]

 [[0.002 0.001]
  [0.003 0.001]
  [0.004 0.001]]]
---Actual---
[[[0.3   0.027]
  [0.4   0.064]
  [0.5   0.125]]

 [[0.4   0.064]
  [0.5   0.125]
  [0.6   0.216]]

 [[0.5   0.125]
  [0.6   0.216]
  [0.7   0.343]]

 [[0.6   0.216]
  [0.7   0.343]
  [0.8   0.512]]

 [[0.7   0.343]
  [0.8   0.512]
  [0.9   0.729]]]


In [ ]:
# Absence of this encoding vector differentiates the regular LSTM network for reconstruction from an LSTM Autoencoder